In [1]:
import flammkuchen as fl
import numpy as np
import time
import seaborn as sns
import pandas as pd
import os
import matplotlib.pyplot as plt
from path import Path
%matplotlib inline

In [2]:
save_path = Path(r"E:\test")

In [3]:
def create_random_data(chunk_size=10, planes=10, roi_x=300, roi_y=300):
    data = np.random.randint(low=0, high=65535, size=[chunk_size, planes, roi_x, roi_y], dtype=np.uint16)
    return data

In [4]:
def save(data, dialog=False, i_file=1, compression_level=1):
    start = time.time_ns()
    fl.save(
        save_path / "{:04d}.h5".format(i_file),
        {"stack_4D": data},
        compression=("blosc", compression_level)
    )
    end = time.time_ns()
    if dialog:
        print("Flammkuchen took {:.2f} seconds to save an array of shape {}".format((end - start) / 1e9, data.shape))
    size = os.path.getsize(save_path /"{:04d}.h5".format(i_file)) / 1048576
    return (end - start) / 1e9, size

In [5]:
def run(n_samples=1, compression_level=1, chunk_sizes=np.arange(1, 11), voxels=(30, 1024, 1024), dialog=False):
    times = np.zeros([len(chunk_sizes), n_samples])
    sizes = np.zeros([len(chunk_sizes), n_samples])
    for i_size, chunk_size in enumerate(chunk_sizes):
        for epoch in range(n_samples):
            data = create_random_data(chunk_size,*voxels)
            times[i_size, epoch], sizes[i_size, epoch] = save(data, i_file=epoch, compression_level=compression_level)
        if dialog:
            print("Data generated for chunk {}".format(i_size + 1))
    return (times, sizes)

In [ ]:
# These are the relevant parameters:
n_samples = 5
blosc_compressions = np.arange(5, 10)
chunk_sizes = np.arange(1, 11)
voxels=(30, 1024, 1024)

for compression in blosc_compressions:
    
    data = run(n_samples=n_samples, compression_level=compression, chunk_sizes=chunk_sizes, voxels=voxels)

    times_pd = pd.DataFrame(data[0])
    times_pd = times_pd.transpose()

    sizes_pd = pd.DataFrame(data[1])
    sizes_pd = sizes_pd.transpose()

    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 7))
    xi = np.arange(1, len(chunk_sizes) + 1)

    time_plot = sns.catplot(data=times_pd, ax=ax1)
    ax1.set(xlabel='Volumes per file', ylabel='Saving time (s)')
    ax1.set_xticklabels(xi)
    plt.clf()

    sns.catplot(data=sizes_pd, ax=ax2)
    ax2.set(xlabel='Volumes per file', ylabel='Size on disk (MB)')
    ax2.set_xticklabels(xi)
    plt.clf()
    
    fig.suptitle("Blosc compression: {}".format(compression), fontsize=30)

    fig.savefig("saving_test_flammkuchen_blosc_{}.png".format(compression), dpi=100)